In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random

from pprint import pprint
from ipywidgets import Video

from PIL import Image
from PIL.ExifTags import TAGS

In [91]:
video_path = './data/12.mp4'
write_path = './movie.avi'

In [92]:
traffic = cv2.VideoCapture(video_path)
if traffic.isOpened():
    print('Film wczytany!')

traffic_width = int(traffic.get(3))
traffic_height = int(traffic.get(4))

print(traffic_height, traffic_width)

traffic_fps = traffic.get(cv2.CAP_PROP_FPS)
print(traffic_fps)

Film wczytany!
720 1280
30.0


In [93]:
Video.from_file(video_path)

Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

In [94]:
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)
lk_params = dict(winSize=(15,15),
                 maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))

In [95]:
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, old_frame = traffic.read()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
mask = np.zeros_like(old_frame)

In [96]:
traffic_optical_flow = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'DIVX'), traffic_fps, (traffic_width, traffic_height))

traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
while traffic.isOpened():
    ret, frame = traffic.read()
    if ret:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is not None:
            good_new = p1[st==1]
            good_old = p0[st==1]

        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)

        traffic_optical_flow.write(cv2.add(frame, mask))
    else:
        break

traffic_optical_flow.release()

In [97]:
!ffmpeg -hide_banner -loglevel error -i movie.avi -y movie.mp4

In [98]:
Video.from_file('./movie.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x04\xb5\xd5mdat\x0…